### 1. code

In [ ]:
### import libraries
import boto3
import json

### remove json file on s3
def s3_rmv_json():
    s3 = boto3.client('s3')
    response = s3.list_objects_v2(Bucket=bucket, Prefix=save_path)
    file_list = []
    if 'Contents' in response:
        for items in response['Contents']:
            file_list.append(items['Key'])
    for i in range(0, len(file_list)):        
        s3.delete_object(Bucket=bucket, Key=file_list[i])
    print(f"deleted files: {save_path}")

### get json file on s3
def s3_get_json():
    data = s3.get_object(Bucket=bucket, Key=json_path)
    json_list = json.loads(data['Body'].read())
    return json_list

### create json dictionary
def create_json_dict(json_list):
    def h_get(key): return h_item.get(key, None)
    json_dict = {'root': [], 'hospital': []}
    for item in json_list:
        for key, value in item.items():
            if key == "ROOT_QUERY":
                for k, v in value.items():
                    if k == 'hospital':
                        r_dict = {
                            k : v
                        }
                json_dict['root'].append(r_dict)
            elif key.startswith("HospitalBase:"):
                h_item = item[key]
                h_dict = {
                    'id' : h_get('id'),
                    'name' : h_get('name'),
                    'category' : h_get('category'),
                    'category_code' : h_get('categoryCode'),
                    'category_code_list' : h_get('categoryCodeList'),
                    'category_count' : h_get('categoryCount'),
                    'description' : h_get('description'),
                    'road_address' : h_get('roadAddress'),
                    'road' : h_get('road'),
                    'rcode' : h_get('rcode'),
                    'virtual_phone' : h_get('virtualPhone'),
                    'phone' : h_get('phone'),
                    'payment_info' : h_get('paymentInfo'),
                    'conveniences' : h_get('conveniences'),
                    'review_setting' : h_get('reviewSettings'),
                    'keywords' : h_get('keywords'),
                    'booking_business_id' : h_get('bookingBusinessId'),
                    'booking_display_name' : h_get('bookingDisplayName'),
                    'visitor_reviews_score' : h_get('visitorReviewsScore'),
                    'visitor_reviews_total' : h_get('visitorReviewsTotal'),
                    'visitor_reviews_text_review_total' : h_get('visitorReviewsTextReviewTotal'),
                    'images' : h_get('images'),
                    'homepages' : h_get('homepages'),
                    'booking_url' : h_get('naverBookingUrl'),
                    'talktalk_url' : h_get('talktalkUrl'),
                    'coordinate' : h_get('coordinate'),
                }
                json_dict['hospital'].append(h_dict)
    return json_dict 

### get json dictionary
def s3_get_json_dict():
    json_list = s3_get_json()
    json_dict = create_json_dict(json_list)
    json_output = json.dumps(json_dict, ensure_ascii=False)
    path = f"{save_path}/json_file.json"
    s3.put_object(Bucket=bucket, Key=path, Body=json_output.encode('utf-8'))
    print(f"uploaded file: {path}")

### 2. run

In [ ]:
### prep
bucket = bucket
json_path = json_path
save_path = save_path
s3 = boto3.client('s3')

In [ ]:
### remove json file
s3_rmv_json()

In [ ]:
### get json dictionary
s3_get_json_dict()